In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../')

data_path = os.path.join(root_dir, 'datadrive/bailian')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'pos/train.csv')
valid_path = os.path.join(data_path, 'pos/valid_small.csv')

custom_path = os.path.join(data_path, 'pos/custom.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
config_file = os.path.join(model_dir, 'pos.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')



In [3]:
from blnlp.pos import PosTagger
tagger = PosTagger()


In [7]:
dataloader = tagger.learner.data

dataloader.reload_dl(custom_path)
dataloader.reload_dl(valid_path, for_train=False)


[2019-03-30 18:31:34 common.py:wrapper:140 DEBUG] get_data cost 0.025548s


[2019-03-30 18:31:35 common.py:wrapper:140 DEBUG] get_data cost 0.978976s


In [4]:
tagger.delete_userdict()
tagger.load_userdict('./untitled.txt')
text = '百炼成钢'
tagger.lexerCustom(text)

2019-04-10 15:58:15,515 DEBUG: get_data cost 0.092432s


2019-04-10 15:58:15,609 DEBUG: single_example_for_predict cost 0.186795s


[('百炼成钢', 'sdsd')]

In [10]:
text_list = [
    '百炼成钢',
    '\n',
    '好人是好人，\n坏人是坏人'
]

tagger.lexerCustom(text_list)

2019-04-10 16:03:09,942 DEBUG: get_data cost 0.027425s


2019-04-10 16:03:10,049 DEBUG: text_array_for_predict cost 0.135121s


[[('百炼成钢', 'nt')],
 [('\n', 'w')],
 [('好人', 'n'),
  ('是', 'v'),
  ('好人', 'n'),
  ('，', 'w'),
  ('\n', 'w'),
  ('坏人', 'n'),
  ('是', 'v'),
  ('坏人', 'n')]]

## 添加自定义词性标注结果集

input: 中文/n 博大精深/a  

output: n_B n_I a_B a_I

In [6]:
def tag_sent_convter(pos_sent, delimiter='△△△'):
    '''
        将词性标注句子转成bert训练的输入格式
    '''
    import re
    import json
    
    p = re.compile(r'(.+?)/(?:([a-z]{1,2})(?:$| ))')
    seg_file = os.path.join(data_path, 'final_baidu-23w.txt')
    
    line = pos_sent.strip()
    if not line:
        return
    
    words = []
    flags = []
    
    replace_chars = [
        '\x97',
        '\uf076',
        "\ue405",
        "\ue105",
        "\ue415",
        '\x07',
        '\x7f',
        '\u3000',
        '\xa0',
        ' '
    ]
    for word, flag in p.findall(line):
        char_list = [c if c not in replace_chars  else 'unk' for c in list(word)]

        tag_list = [f'B_{flag}'] + [f'I_{flag}']  * (len(char_list) - 1)

        if len(char_list) != len(tag_list):
            print(line)
            print(word, flag)
            print(char_list, tag_list)

        words.extend(char_list)
        flags.extend(tag_list)
    
    if len(words) == 0:
        return
    
    return delimiter.join([
        ' '.join(flags),
        ' '.join(words)
    ])

def save2customfile(pos_sent):
    '''
        保存到自定义文档中，主要用于增量训练反馈数据
    '''
    
    bert_sent = tag_sent_convter(pos_sent)
    if not bert_sent:
        return
    
    with open(custom_path, 'a') as f:
        f.write(bert_sent)
        f.write('\n')
    
    

pos_sents = [
    '周光/nr 明确/ad 否认/v CEO/ti 佟显侨/nr 和/c CTO/ti 衡量/nr 说/v 的/u 罪名/n ，/w 并/c 说会/v 跟/p 投资人/vn 、/w 技术leader/ti 一起/d 出来说明/v',
    '当时/t 其/r 2位/m 创始人/ti 佟显侨/nr 和/c 衡量/nr 推动发出/v 公司公告/n ，/w 把/p 还在/v 东京/ns 代表/v RoadStar/nt 出席/v 活动/n 的/u 另/r 一/m 联合创始人/ti 周光/nr 开除/v 。/w'
]

for pos_sent in pos_sents:
    save2customfile(pos_sent)


            

In [3]:
from tqdm.auto import tqdm
for i in tqdm(range(1000)):
    pass